In [71]:
# Let's import all the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

In [72]:
df = pd.read_csv('train.csv')

In [73]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [74]:
df.drop('Id',axis = 1,inplace=True)

In [75]:
df.shape

(1460, 80)

In [76]:
df.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [77]:
df.columns[df.isnull().sum()>0]

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')

In [78]:
df[['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual','BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2','Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt','GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence','MiscFeature']].isnull().sum()

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [79]:
df.select_dtypes(include=np.number)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,61,0,0,0,0,0,2,2008,208500
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,298,0,0,0,0,0,0,5,2007,181500
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,42,0,0,0,0,0,9,2008,223500
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,35,272,0,0,0,0,2,2006,140000
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0.0,0,0,...,0,40,0,0,0,0,0,8,2007,175000
1456,20,85.0,13175,6,6,1978,1988,119.0,790,163,...,349,0,0,0,0,0,0,2,2010,210000
1457,70,66.0,9042,7,9,1941,2006,0.0,275,0,...,0,60,0,0,0,0,2500,5,2010,266500
1458,20,68.0,9717,5,6,1950,1996,0.0,49,1029,...,366,0,112,0,0,0,0,4,2010,142125


---
## Linear Regression
---

In [80]:
Scaler = MinMaxScaler()

In [81]:
df_linreg = df
numeric_features = df_linreg.select_dtypes(include=np.number).columns

In [82]:
df_linreg[numeric_features] = Scaler.fit_transform(df_linreg[numeric_features])

In [83]:
df_linreg.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,0.235294,RL,0.150685,0.033420,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0.0,NaN,NaN,NaN,0.0,0.090909,0.50,WD,Normal,0.241078
1,0.000000,RL,0.202055,0.038795,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0.0,NaN,NaN,NaN,0.0,0.363636,0.25,WD,Normal,0.203583
2,0.235294,RL,0.160959,0.046507,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0.0,NaN,NaN,NaN,0.0,0.727273,0.50,WD,Normal,0.261908
3,0.294118,RL,0.133562,0.038561,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0.0,NaN,NaN,NaN,0.0,0.090909,0.00,WD,Abnorml,0.145952
4,0.235294,RL,0.215753,0.060576,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0.0,NaN,NaN,NaN,0.0,1.000000,0.50,WD,Normal,0.298709


In [84]:
non_numeric = df_linreg.columns[~df_linreg.columns.isin(df_linreg[numeric_features].columns)]

In [85]:
for column in non_numeric:
    df_linreg = pd.concat([df_linreg,pd.get_dummies(df[column])],axis = 1)

In [86]:
df_linreg.drop(non_numeric,1,inplace=True)
df_linreg.shape


(1460, 289)

In [87]:
df_linreg.isnull().sum()[df_linreg.isnull().sum()>0]

LotFrontage    259
MasVnrArea       8
GarageYrBlt     81
dtype: int64

In [88]:
df_linreg.dropna(inplace=True)

## Test - Train Split

In [89]:
np.random.seed(0)
df_train, df_test = train_test_split(df_linreg, train_size = 0.7, test_size = 0.3, random_state = 100)

In [90]:
y_train = df_train['SalePrice']
X_train = df_train.drop('SalePrice',1)
y_test = df_test['SalePrice']
X_test = df_test.drop('SalePrice',1)

## Linear Regression

In [91]:
X_train_lm = sm.add_constant(X_train)
lr = sm.OLS(y_train,X_train_lm).fit()

In [92]:
(lr.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.946
Model:                            OLS   Adj. R-squared:                  0.925
Method:                 Least Squares   F-statistic:                     43.81
Date:                Tue, 01 Mar 2022   Prob (F-statistic):          6.52e-258
Time:                        11:45:11   Log-Likelihood:                 1777.9
No. Observations:                 784   AIC:                            -3104.
Df Residuals:                     558   BIC:                            -2050.
Df Model:                         225                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
MSSubClass       -0.0086      0.035     -0.248      0.804      -0.077       0.060
LotFrontage       0.0121      0.027      0.454      0.650      -0.040       0.064
LotArea           0.5174      0.081      6.354      0.000       0.357       0.677
OverallQual       0.0739      0.019      3.971      0.000       0.037       0.110
OverallCond       0.0655      0.015      4.423      0.000       0.036       0.095
YearBuilt         0.0655      0.023      2.823      0.005       0.020       0.111
YearRemodAdd      0.0078      0.007      1.163      0.246      -0.005       0.021
MasVnrArea        0.0593      0.017      3.443      0.001       0.025       0.093
BsmtFinSF1        0.1459      0.032      4.502      0.000       0.082       0.210
BsmtFinSF2        0.0250      0.024      1.036      0.301      -0.022       0.072
BsmtUnfSF         0.0179      0.014      1.265      0.207      -0.010       0.046
TotalBsmtSF       0.1476      0.034      4.330      0.000       0.081       0.215
1stFlrSF          0.1492      0.032      4.647      0.000       0.086       0.212
2ndFlrSF          0.1077      0.023      4.767      0.000       0.063       0.152
LowQualFinSF      0.0137      0.026      0.520      0.603      -0.038       0.065
GrLivArea         0.1659      0.025      6.619      0.000       0.117       0.215
BsmtFullBath      0.0111      0.011      0.968      0.334      -0.011       0.034
BsmtHalfBath  -8.206e-05      0.011     -0.007      0.994      -0.023       0.022
FullBath          0.0288      0.014      2.119      0.035       0.002       0.056
HalfBath          0.0138      0.008      1.625      0.105      -0.003       0.030
BedroomAbvGr     -0.0538      0.023     -2.381      0.018      -0.098      -0.009
KitchenAbvGr     -0.0817      0.046     -1.769      0.077      -0.172       0.009
TotRmsAbvGrd      0.0271      0.022      1.217      0.224      -0.017       0.071
Fireplaces        0.0283      0.016      1.795      0.073      -0.003       0.059
GarageYrBlt      -0.0053      0.013     -0.396      0.693      -0.031       0.021
GarageCars        0.0355      0.017      2.140      0.033       0.003       0.068
GarageArea        0.0084      0.021      0.395      0.693      -0.033       0.050
WoodDeckSF        0.0147      0.010      1.465      0.143      -0.005       0.034
OpenPorchSF       0.0200      0.014      1.467      0.143      -0.007       0.047
EnclosedPorch     0.0039      0.014      0.282      0.778      -0.023       0.031
3SsnPorch         0.0458      0.020      2.263      0.024       0.006       0.086
ScreenPorch       0.0160      0.012      1.353      0.177      -0.007       0.039
PoolArea          0.0898      0.021      4.240      0.000       0.048       0.131
MiscVal           0.0517      0.211      0.245      0.806      -0.363       0.466
MoSold            0.0047      0.005      0.880   

In [93]:
model = LinearRegression()

In [94]:
model.fit(X_train_lm,y_train)
print(model.score(X_train_lm,y_train))

0.9464219419690233


In [105]:
y_test_pred = model.predict(X_test)

In [108]:
model.score(sm.add_constant(X_test),y_test)

-4.502032961444353e+19

## Ridge

In [109]:
for i in [0,0.0001,0.001,0.01,0.1,1,10,100]:
    ridge = Ridge(alpha=i)
    ridge.fit(X_train_lm,y_train)
    y_test_pred = ridge.predict(sm.add_constant(X_test))
    print("lambda = {}, r2 score = ".format(i) + str(r2_score(y_test, y_test_pred))) 

lambda = 0, r2 score = -6.267793847783145e+26
lambda = 0.0001, r2 score = 0.8169182661652024
lambda = 0.001, r2 score = 0.8174297674472316
lambda = 0.01, r2 score = 0.8218295760165091
lambda = 0.1, r2 score = 0.8352607591189984
lambda = 1, r2 score = 0.8191394660504794
lambda = 10, r2 score = 0.7929764480239943
lambda = 100, r2 score = 0.7144805425859218


## Lasso 

In [110]:
values = np.linspace(0,0.001,100)
results = []
for i in values:
    lasso = Lasso(alpha=i)
    lasso.fit(X_train_lm,y_train)
    y_test_pred = lasso.predict((sm.add_constant(X_test)))
    results.append(r2_score(y_test,y_test_pred))
#     print("lambda = {}, r2 score = ".format(i) + str(r2_score(y_test, y_test_pred))) 
#     print('{:0.1f}%'.format((lasso.coef_==0).sum()/288*100))
#     print('-'*50)


In [113]:
max(results)

0.8340197176261149

In [111]:
values[results.index(max(results))]

9.090909090909092e-05

In [114]:
lasso = Lasso(alpha=0.0001)
lasso.fit(X_train_lm,y_train)
y_test_pred = lasso.predict((sm.add_constant(X_test)))
print(r2_score(y_test,y_test_pred))

0.8340034701705483


Obtained a R2 score of 0.835

### Exploring features

In [122]:
(lasso.coef_==0).sum()

173

In [115]:
coeff_df = pd.DataFrame(data={'Columns':X_train_lm.columns,'Coeff':lasso.coef_})

In [116]:
coeff_df.sort_values('Coeff',ascending=False,inplace=True)

In [117]:
coeff_df.tail(5)

,Columns,Coeff
70,Edwards,-0.014592
41,Grvl,-0.018333
129,Tar&Grv,-0.019955
36,C (all),-0.020569
124,ClyTile,-0.578775
